In [1]:

# coding: utf-8
epoch=0

# dependencies
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import time
import data_utils
import matplotlib.pyplot as plt
from __check import check_sentence

['you', '-', 'I', 'to', 'the', 's', 'a', 't', 'of', 'it', 'me', 'is', 'and', "I'", 'your', 'You', 'that', 'in', 'are', 'this', 'be', 'on', 're', 'What', 'for', 'have', 'my', 'm', "don'", 'here', 'know', 'not', 'do', 'il', 'go', 'we', 'no', 'what', 'think', 'all', 'one', 'with', 'Harry', 'can', 'was', 'he', 'just', 'up', 'get', 'so', 'but', 'school', 'like', "it'", 'him', 'The', 've', 'about', "It'", 'right', 'out', 'come', 'at', "You'", 'Oh', 'her', 'will', 'Dobby', 'Big', 'if', "you'", 'brother', 'Why', 'been', 'back', 'now', 'see', 'how', 'How', 'by', 'from', 'them', 'did', 'sir', 'time', 'If', 'there', 'No', 'who', 'And', 'Potter', "can'", 'only', 'tell', 'Well', '--', 'our', 'going', 'too', 'they', 'Yes', 'This', 'Come', 'has', 'were', 'We', 'But', 'want', 'take', 'even', 'Do', 'never', 'really', 'look', "didn'", 'Hey', 'day', "That'", 'way', 'Are', 'when', 'am', 'well', "What'", 'us', "Don'", 'little', 'two', 'home', 'told', 'must', 'So', 'years', "we'", 'his', 'It', 'said', 'say'

1962
50


In [2]:
X, Y, en_word2idx, en_idx2word, en_vocab, zh_word2idx, zh_idx2word, zh_vocab = data_utils.read_dataset('en_n_zh.pkl')

In [3]:
print(len(Y))
#print(_check_how.f())

1097


In [4]:
# inspect data
print('Sentence in English - encoded:', X[0])
print('Sentence in Chinese - encoded:', Y[0])
print('Decoded:\n------------------------')

for i in range(len(X[2])):
    print(en_idx2word[X[2][i]], end=' ')
    
print('\n')

for i in range(len(Y[2])):
    print(zh_idx2word[Y[2][i]], end=' ')

Sentence in English - encoded: [917, 13, 6, 1452, 5, 576]
Sentence in Chinese - encoded: [782, 8, 54, 568, 7, 1303]
Decoded:
------------------------
The area is too important for him 

這 碼頭 對 他 很重 要 

In [5]:
# data processing
X_en2zh,Y_zh2en,Y_en2zh,X_zh2en = train_test_split(X,Y,0.5)
# data padding
def data_padding(x, y, length = 16):
    for i in range(len(x)):
        x[i] = x[i] + (length - len(x[i])) * [en_word2idx['<pad>']]
        y[i] = [zh_word2idx['<go>']] + y[i] + [zh_word2idx['<eos>']] + (length-len(y[i])) * [zh_word2idx['<pad>']]
data_padding(X_en2zh, Y_en2zh)
data_padding(X_zh2en, Y_zh2en)
#print(Y)

TypeError: Singleton array array(0.5) cannot be considered a valid collection.

In [ ]:
X_en2zh_train,  X_en2zh_test, Y_en2zh_train, Y_en2zh_test = train_test_split(X_en2zh, Y_en2zh, test_size = 0.1)
X_zh2en_train, X_zh2en_test, Y_zh2en_train, Y_zh2en_test = train_test_split(X_zh2en,Y_zh2en, test_size= 0.1)

In [ ]:
#X_train

In [ ]:
input_seq_len = 16
output_seq_len = 18
en_vocab_size = len(en_vocab) + 2 # + <pad>, <ukn>
zh_vocab_size = len(zh_vocab) + 4 # + <pad>, <ukn>, <eos>, <go>

In [ ]:
print(en_vocab_size)

In [ ]:
en_encoder_inputs = tf.placeholder(dtype = tf.int32, shape = [None,input_seq_len], name = 'encoder_en2zh')
en_decoder_inputs = tf.placeholder(dtype = tf.int32, shape = [None,output_seq_len], name = 'decoder_en2zh')

zh_encoder_inputs = tf.placeholder(dtype = tf.int32, shape = [None,input_seq_len], name = 'encoder_zh2en')
zh_decoder_inputs = tf.placeholder(dtype = tf.int32, shape = [None,output_seq_len], name = 'decoder_zh2en')

In [ ]:
targets_en2zh = tf.placeholder(dtype = tf.int32, shape = [None,output_seq_len], name = 'target_en2zh')
# add one more target
target_en2zh_weights = tf.placeholder(dtype = tf.int32, shape = [None,output_seq_len], name = 'target_en2zh_weights')
targets_zh2en = tf.placeholder(dtype = tf.int32, shape = [None,output_seq_len], name = 'target_zh2en')
# add one more target
target_zh2en_weights = tf.placeholder(dtype = tf.int32, shape = [None,output_seq_len], name = 'target_zh2en_weights')

In [ ]:
# output projection
size = 512
w_t = tf.get_variable('proj_w', [zh_vocab_size, size], tf.float32)
b = tf.get_variable('proj_b', [zh_vocab_size], tf.float32)
w = tf.transpose(w_t)
output_projection = (w, b)

In [ ]:
print(tf.reduce_mean(w))

In [ ]:
with tf.name_scope("en2zh"):
    outputs_zh, states_zh = tf.contrib.legacy_seq2seq.embedding_attention_seq2seq(
                                                en_encoder_inputs,
                                                en_decoder_inputs,
                                                tf.contrib.rnn.BasicLSTMCell(size),
                                                num_encoder_symbols = en_vocab_size,
                                                num_decoder_symbols = zh_vocab_size,
                                                embedding_size = 100,
                                                feed_previous = False,
                                                output_projection = output_projection,
                                                dtype = tf.float32)

In [ ]:
with tf.name_scope("zh2en"):
    outputs_en, states_en = tf.contrib.legacy_seq2seq.embedding_attention_seq2seq(
                                                zh_encoder_inputs,
                                                zh_decoder_inputs,
                                                tf.contrib.rnn.BasicLSTMCell(size),
                                                num_encoder_symbols = zh_vocab_size,
                                                num_decoder_symbols = en_vocab_size,
                                                embedding_size = 100,
                                                feed_previous = False,
                                                output_projection = output_projection,
                                                dtype = tf.float32)

In [ ]:
# define our loss function

# sampled softmax loss - returns: A batch_size 1-D tensor of per-example sampled softmax losses
def sampled_loss_en2zh(labels, logits):
    return tf.nn.sampled_softmax_loss(
                        weights = w_t,
                        biases = b,
                        labels = tf.reshape(labels, [-1, 1]),
                        inputs = logits,
                        num_sampled = 512,
                        num_classes = zh_vocab_size)
def sampled_loss_zh2en(labels, logits):
    return tf.nn.sampled_softmax_loss(
                        weights = w_t,
                        biases = b,
                        labels = tf.reshape(labels, [-1, 1]),
                        inputs = logits,
                        num_sampled = 512,
                        num_classes = en_vocab_size)

## define loss using tf.cond

In [ ]:
# Weighted cross-entropy loss for a sequence of logits
loss_en2zh_warmup = tf.contrib.legacy_seq2seq.sequence_loss(outputs_zh, targets_en2zh, target_en2zh_weights, softmax_loss_function = sampled_loss_en2zh)


loss_zh2en_warmup = tf.contrib.legacy_seq2seq.sequence_loss(outputs_en, targets_zh2en, target_zh2en_weights, softmax_loss_function = sampled_loss_zh2en)

loss_odd=tf.contrib.legacy_seq2seq.sequence_loss(outputs_zh,zh_encoder_inputs , softmax_loss_function = sampled_loss_en2zh)
loss_no_odd=tf.contrib.legacy_seq2seq.sequence_loss(outputs_en, en_encoder_inputs, softmax_loss_function = sampled_loss_en2zh)
#lack target_weight

In [ ]:
# let's define some helper functions

# simple softmax function
def softmax(x):
    n = np.max(x)
    e_x = np.exp(x - n)
    return e_x / e_x.sum()

In [ ]:
# feed data into placeholders
#def feed_dict(x, y, batch_size = 64):
def batch_generator_double(dataset1,dataset2,batch_size=50)
    current_number=0
    while 1:
        if current_number+batch_size>=len(dataset1):
            current_number=0
        yield dataset1[current:current_number+batch_size],dataset2[current:current+batch_size]
        current_number+=batch_size
def batch_generator_single(dataset,batch_size=50)
    current_number=0
    while 1:
        if current_number+batch_size>=len(dataset):
            current_number=0
        yield dataset[current:current_number+batch_size]
        current_number+=batch_size
def weight_calculator(batch):
    weight=[]
    for i in range(len(batch)):
        if batch[i] is '<pad>':
            weight.append(0.0)
        else:
            weight.append(1.0)
    return weight
def del_go_n_eos(batch):
    for i in range(len(batch)):
        if batch[i] is '<go>' or batch[i] is '<eos>':
            del batch[i]
    
    
    '''
    feed = {}
    
    idxes = np.random.choice(len(x), size = batch_size, replace = False)
    
    for i in range(input_seq_len):
        feed[encoder_inputs[i].name] = np.array([x[j][i] for j in idxes], dtype = np.int32)
        
    for i in range(output_seq_len):
        feed[decoder_inputs[i].name] = np.array([y[j][i] for j in idxes], dtype = np.int32)
        
    feed[targets[len(targets)-1].name] = np.full(shape = [batch_size], fill_value = zh_word2idx['<pad>'], dtype = np.int32)
    
    for i in range(output_seq_len-1):
        batch_weights = np.ones(batch_size, dtype = np.float32)
        target = feed[decoder_inputs[i+1].name]
        for j in range(batch_size):
            if target[j] == zh_word2idx['<pad>']:
                batch_weights[j] = 0.0
        feed[target_weights[i].name] = batch_weights
        
    feed[target_weights[output_seq_len-1].name] = np.zeros(batch_size, dtype = np.float32)
    
    return feed
    '''

In [ ]:
# decode output sequence
def decode_output(output_seq,idx2word=zh_id2word):
    words = []
    for i in range(output_seq_len):
        smax = softmax(output_seq[i])
        idx = np.argmax(smax)
        words.append(idx2word[idx])
    return words

In [ ]:
# ops and hyperparameters
learning_rate = 5e-3
batch_size = 64
steps = 1000

# ops for projecting outputs
outputs_proj_en2zh = [tf.matmul(outputs_en2zh[i], output_projection[0]) + output_projection[1] for i in range(output_seq_len)]
outputs_proj_zh2en = [tf.matmul(outputs_zh2en[i], output_projection[0]) + output_projection[1] for i in range(output_seq_len)]

# training op
with tf.name_scope("en2zh"):
    optimizer_en2zh_warmup = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_en2zh_warmup)
with tf.name_scope("zh2en"):
    optimizer_zh2en_warmup = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_zh2en_warmup)


with tf.name_scope("en2zh"):
    optimizer_odd_en2zh = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_odd)
    optimizer_no_odd_en2zh = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_no_odd)
with tf.name_scope('zh2en'):
    optimizer_odd_zh2en = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_odd)
    optimizer_no_odd_zh2en = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_no_odd)
# init op
init = tf.global_variables_initializer()

# forward step
def forward_step(sess, feed,outputs_proj):
    output_sequences = sess.run(outputs_proj, feed_dict = feed)
    return output_sequences

# training step
def backward_step(sess, feed,optimizer):
    sess.run(optimizer, feed_dict = feed)

In [ ]:
next_batch_en2zh=batch_generator_double(X_en2zh_train,Y_en2zh_train)
next_bacth_zh2en=batch_generator_double(X_zh2en_train,Y_zh2en_train)

next_batch_odd=batch_generator_single(X_en2zh)
next_batch_no_odd=batch_generator_single(X_zh2en)

In [ ]:
# let's train the model
n_epoch=1000
batch_size=50
# we will use this list to plot losses through steps
losses = []

# save a checkpoint so we can restore the model later 
saver = tf.train.Saver()

print('------------------TRAINING------------------')

with tf.Session() as sess:
    sess.run(init)
    
    t = time.time()
    for step in range(n_epoch/10):
        batch_en2zh=next(next_batch_en2zh)
        for i in range(len(batch_en2zh[1])):
            batch_en2zh[1][i]=batch_en2zh[:len(bacth_en2zh[1][i])-2]+'<pad>'
        batch_zh2en=next(next_batch_zh2en)
        for i in range(len(batch_zh2en[1])):
            batch_zh2en[1][i]=batch_zh2en[:len(bacth_zh2en[1][i])-2]+'<pad>'
        backward_step(sess,{X_en2zh:batch_en2zh[0],target_en2zh:batch_en2zh[1]},optimizer_en2zh_warmup)
        backward_step(sess,{X_en2zh:batch_zh2en[0],Y_zh2en:batch_zh2en[1]},optimizer_zh2en_warmup)
        if step % 5 == 4 or step == 0:
            loss_value_en2zh = sess.run(loss_en2zh_warmup, feed_dict = {X_en2zh_test})
            loss_value_zh2en = sess,run(loss_zh2en_warmup, X_zh2en_test)
            print('step: {}, loss: {}'.format(step, loss_value))
            #losses.append(loss_value)
        # translate
            if step%50==0:
                output_sequences = forward_step(sess,feed)
                #print(output_sequences)
                output=[]
            # decode seq.
                for i in range(len(output_sequences[0])):
                    #print '{}.\n--------------------------------'.format(i+1)
                    ouput_seq = [output_sequences[j][i] for j in range(output_seq_len)]
                    #decode output sequence
                    words = decode_output(ouput_seq)
                    word=[zh_word2idx[word] for word in words]
                    output.append(words)
                    #print(len(output))
                print(output)
                print(check_sentence(output,zh_vocab,zh_word2idx))
        if step % 20 == 19:
            saver.save(sess, './data/warmup')
            print('Checkpoint is saved')
            
    print('Training time for {} steps: {}s'.format(steps, time.time() - t))

In [ ]:
'''
en_encoder_inputs = tf.placeholder(dtype = tf.int32, shape = [None,input_seq_len], name = 'encoder_en2zh')
en_decoder_inputs = tf.placeholder(dtype = tf.int32, shape = [None,output_seq_len], name = 'decoder_en2zh')

zh_encoder_inputs = tf.placeholder(dtype = tf.int32, shape = [None,input_seq_len], name = 'encoder_zh2en')
zh_decoder_inputs = tf.placeholder(dtype = tf.int32, shape = [None,output_seq_len], name = 'decoder_zh2en')
'''
with tf.Session() as sess:
        saver.restore(sess,'./data/warmup')
        for i in range(n_epoch):
            epoch=i
            batch_en2zh=next(next_batch_en2zh)
            for i in range(len(batch_en2zh[1])):
                batch_en2zh[1][i]=batch_en2zh[:len(bacth_en2zh[1][i])-2]+'<pad>'
            batch_zh2en=next(next_batch_zh2en)
            for i in range(len(batch_zh2en[1])):
                batch_zh2en[1][i]=batch_zh2en[:len(bacth_zh2en[1][i])-2]+'<pad>'
            if epoch % 2 ==0:
                
                output_sequences = forward_step(sess,{X_en2zh : batch_en2zh[0]})
                #print(output_sequences)
                output=[]
            # decode seq.
                for i in range(len(output_sequences[0])):
                    #print '{}.\n--------------------------------'.format(i+1)
                    ouput_seq = [output_sequences[j][i] for j in range(output_seq_len)]
                    #decode output sequence
                    words = decode_output(ouput_seq)
                    #word=[zh_word2idx[word] for word in words]
                    output.append(words)
                    #print(len(output))
                del_go_n_eos(output)
                for i in range(len(batch_en2zh[0])):
                    batch_en2zh[1][i]=batch_en2zh[:len(bacth_en2zh[0][i])-2]+'<pad>'
                backward_step(sess,{X_en2zh:output,target_en2zh:batch_en2zh[0]},optimizer_odd_en2zh)
                backward_step(sess,{X_en2zh:output,target_en2zh:batch_en2zh[0]},optimizer_odd_zh2en)
            else:
                output_sequences = forward_step(sess,{X_en2zh : batch_zh2en[0]})
                #print(output_sequences)
                output=[]
            # decode seq.
                for i in range(len(output_sequences[0])):
                    #print '{}.\n--------------------------------'.format(i+1)
                    ouput_seq = [output_sequences[j][i] for j in range(output_seq_len)]
                    #decode output sequence
                    words = decode_output(ouput_seq)
                    #word=[en_word2idx[word] for word in words]
                    output.append(words)
                    #print(len(output))
                del_go_n_eos(output)
                for i in range(len(batch_zh2en[0])):
                    batch_en2zh[1][i]=batch_zh2en[:len(bacth_zh2en[0][i])-2]+'<pad>'
                backward_step(sess,{X_zh2en:output,target_zh2en:batch_zh2en[0]},optimizer_odd_zh2en)
                backward_step(sess,{X_zh2en:output,target_zh2en:batch_zh2en[0]},optimizer_odd_en2zh)